In [ ]:
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
from analysis.lib.m2.ssro import ssro, sequence
import scipy.fftpack
import win32com.client #imports the pywin32 library
%matplotlib inline

In [ ]:
# def set_limits():
#     ind = np.argmin(np.abs(freq[0][0] - 1.72e9))
#     plt.xlim(1.64e9, 1.8e9)
#     plt.ylim(0, 1.2*fft[0][0, ind])
def arg_value(t, t_0):
    ind = np.argmin(np.abs(t - t_0))
    return ind
    

def fourier(signal, time):
    """Returns the Fouier transform of a single signal with time, including the negative frequencies."""
    t_step = (time[-1] - time[0])/time.size
    fft = scipy.fftpack.fft(signal) * t_step
    freq = scipy.fftpack.fftfreq(time.size, t_step)
    return np.abs(np.fft.fftshift(fft)), np.fft.fftshift(freq)

def fourier_pos_freq(signal, time):
    """Returns the Fouier transform of a single signal with time."""
    t_step = (time[-1] - time[0])/time.size
    fft = scipy.fftpack.fft(signal) * t_step
    freq = scipy.fftpack.fftfreq(time.size, t_step)
    n = len(fft)
    fft = fft[0:n//2]
    freq = freq[0:n//2]
    return np.abs(fft), freq

def hermite_pulse_env(env_amplitude, T_herm, mu=0):
    t = np.linspace(-25e-8,25e-8,int(1e4)) + mu
    env = env_amplitude*(1-0.956*((t-mu)/T_herm)**2)*np.exp(-((t-mu)/T_herm)**2)
    return env, t

def hermite_pulse(env_amplitude, T_herm, freqency, mu=0):
    env, t = hermite_pulse_env(env_amplitude, T_herm, mu=0)
    hermite_pulse = env * np.sin(t*2*np.pi*frequency)
    return hermite_pulse

def path_folder(disk='D', when='latest', multiple_msmt=False, older_than=None, newer_than=None):
    """Returns a list with string with the path to the measurements."""
    if disk == 'D':
        data_folder = None
    else:
        data_folder = '{}:/data'.format(disk)
    if multiple_msmt == False:
        if when == 'latest':
            msmt_folder = tb.latest_data('mw_pulse_msmt', folder=data_folder)
        else:
            msmt_folder = tb.data_from_time(when, folder=data_folder)
    else:
        msmt_folder = tb.latest_data('mw_pulse_msmt', folder='X:/data',
                                     newer_than=newer_than, older_than=older_than, return_all=True)
    return msmt_folder

class mw_msmt:
    def __init__(self, disk='D', when='latest'):
        msmt_folder = path_folder(disk, when)
        f = sequence.SequenceAnalysis(msmt_folder, hdf5_mode='r')
        self.number_of_pulses = int(f.g.attrs['multiplicity'])
        self.points_per_pulse = int(1e6 // self.number_of_pulses)
        #For pulses of 124ns
#         self.points_per_pulse += 520
#         self.number_of_pulses -= 1
        ####
        self.frequency = f.g.attrs['mw_frq']
        self.length = f.g.attrs['Hermite_pi_length']
        self.t_herm = 0.1667*self.length
        self.t = np.zeros((self.number_of_pulses, self.points_per_pulse))
        self.amplitude = np.zeros((self.number_of_pulses, self.points_per_pulse))
        self.middle = np.zeros(self.number_of_pulses)
        self.offset = np.zeros(self.number_of_pulses)
#         self.imod = np.array(f.g['pulses']['MW_Imod'])
#         self.awg_time = np.array(f.g['pulses']['time'])

        waveform = np.array(f.g['oscilloscope_msmt']['data'])
        if disk=='D':
            self.timestamp = msmt_folder[18:33]
        else:
            self.timestamp = msmt_folder[8:23]
        for pulse in range(self.number_of_pulses):
            self.t[pulse] = waveform[0][pulse*self.points_per_pulse:(pulse+1)*self.points_per_pulse]
            self.amplitude[pulse] = waveform[1][pulse*self.points_per_pulse:(pulse+1)*self.points_per_pulse]
            self.middle[pulse] = self.t[pulse][np.argmax(self.amplitude[pulse])]
#             self.offset[pulse] = self.amplitude[pulse][0:arg_value(self.t[pulse], self.t[pulse][np.argmax(self.amplitude)]-.75*self.length)].mean()
        self.offset = np.mean(self.amplitude[:][0:1000])
        self.amplitude -= self.offset # Corrects for a DC ofset of the signal
            
            
    @property
    def mean_fft(self):
        mean = np.mean(self.fft, axis=0)
        return mean
    
    @property
    def std_fft(self):
        std = np.std(self.fft, axis=0)
        return std
    

class pulse_msmt(mw_msmt):
    def __init__(self, disk='D', when='latest'):
        mw_msmt.__init__(self, disk, when)
        self.fft = np.zeros((self.number_of_pulses, self.points_per_pulse//2))
        self.freq = np.zeros((self.number_of_pulses, self.points_per_pulse//2))
        for pulse in range(self.number_of_pulses):
            self.fft[pulse], self.freq[pulse] = fourier_pos_freq(self.amplitude[pulse], self.t[pulse])
        
    def integral_fft(self, lower_bound=None, upper_bound=None):
        integral = np.zeros(self.number_of_pulses)
        if lower_bound==None and upper_bound==None:
            for pulse in range(self.number_of_pulses):
                integral[pulse] = scipy.integrate.trapz(self.fft[pulse], self.freq[pulse])
        else:
            
            for pulse in range(self.number_of_pulses):
                ind_lower = arg_value(self.freq[pulse], lower_bound)
                ind_upper = arg_value(self.freq[pulse], upper_bound)
                integral[pulse] = scipy.integrate.trapz(self.fft[pulse][ind_lower:ind_upper], 
                                                        self.freq[pulse][ind_lower:ind_upper])
        return integral

class enveloppe_msmt(mw_msmt):
    def __init__(self, disk='D', when='latest'):
        mw_msmt.__init__(self, disk, when)
        self.frequency = 0
        self.fft = np.zeros((self.number_of_pulses, self.points_per_pulse))
        self.freq = np.zeros((self.number_of_pulses, self.points_per_pulse))
        for pulse in range(self.number_of_pulses):
            self.fft[pulse], self.freq[pulse] = fourier(self.amplitude[pulse], self.t[pulse])

class sequential_pulse_msmts:
    def __init__(self, newer_than, older_than, disk='D'):
        msmt_folders = path_folder(disk, newer_than=newer_than, older_than=older_than, multiple_msmt=True)
        self.number_of_msmts = len(msmt_folders)
        self.msmts = []
        for msmt_folder in msmt_folders:
            when = tb.timestamp_from_datetime(tb.get_datetime_from_folder(msmt_folder))
            self.msmts.append(pulse_msmt(disk=disk, when=when))

In [ ]:
# a = enveloppe_msmt('X')
# b = sequential_pulse_msmts(disk='X',newer_than='20180321182000', older_than='20180322110000')
# a = enveloppe_msmt('X', when='20180323091723')
# a = enveloppe_msmt('Z')
# a = enveloppe_msmt('Z', when='20180326125434')
# lt_4_rf_source = pulse_msmt('X', when='20180320144111')
# lt_4_amplifier = pulse_msmt('X', when='20180321111625')
# lt_4_switch = pulse_msmt('X', when='20180321120058')
lt_4_cryo = pulse_msmt('X', when='20180321134817')

In [ ]:
msmt = lt_4_switch
pulse = 30

In [ ]:
(msmt.length-100e-9)/50e-12

# Overview of the measurement

In [ ]:
plt.figure(figsize=(10,8))
for pulse in range(msmt.number_of_pulses):
    plt.plot(msmt.t[pulse], msmt.amplitude[pulse])
plt.xlabel('t(s)')
plt.ylabel('U(V)')
plt.title('Overview of the whole measurement, timestamp: {}'.format(msmt.timestamp))
plt.show()

# Analysis of single pulse

In [ ]:
# msmt = enveloppe_msmt('Z')

# Plot waveform
plt.figure(figsize=(8,6))
plt.plot(msmt.t[pulse], msmt.amplitude[pulse], label='Data')
# Plot ideal pulse
middle = (msmt.t[pulse][-1]-msmt.t[pulse][0])/2 + msmt.t[pulse][0]
if msmt.__class__.__name__ == 'enveloppe_msmt':
    env, t = hermite_pulse_env(env_amplitude=np.max(msmt.amplitude), mu=msmt.middle[pulse], T_herm=msmt.t_herm)
    plt.plot(t, env, '--', label='Ideal')
    plt.plot(msmt.awg_time-1.9865e-5, msmt.imod/1.05, label='Send to AWG') #Plot input for the AWG
elif msmt.__class__.__name__ == 'pulse_msmt':
    pass



time_window = 2e-7
plt.xlim(msmt.middle[pulse]-time_window/2, msmt.middle[pulse]+time_window/2)
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
plt.xlabel('s')
plt.ylabel('V')
plt.title('Single Hermite pulse, timestamp : {}'.format(msmt.timestamp))
plt.legend()
plt.show()

# Plot the fourier transform
plt.figure(figsize=(8,5))
plt.plot(msmt.freq[pulse], msmt.fft[pulse], label='Data')
# Plot ideal transform
if msmt.__class__.__name__ == 'enveloppe_msmt':
    fft, freq = fourier(env, t)
    plt.plot(freq, fft, '--', label='Ideal')
elif msmt.__class__.__name__ == 'pulse_msmt':
    pass

plt.title('Fourier transform, timestamp : {}'.format(msmt.timestamp))
plt.xlabel('Frequency [Hz]')
plt.legend()
plt.xlim(msmt.frequency - 1e8, msmt.frequency + 1e8)
plt.show()

# Analysis of the pulses in one measurement

In [ ]:
half_width_integral = 30e6

plt.figure(figsize=(10,8))
for pulse in range(msmt.number_of_pulses):
    plt.plot(msmt.freq[pulse], msmt.fft[pulse], label='{}'.format(pulse))
plt.xlim(msmt.frequency - 1e8, msmt.frequency + 1e8)
plt.axvspan(msmt.frequency-half_width_integral, msmt.frequency+half_width_integral, alpha=0.2, color='red')

plt.xlabel('f (Hz)')
plt.title(' Fourier transforms of {} consecutive pulses , timestamp : {}'.format(msmt.number_of_pulses, msmt.timestamp))
# plt.legend()
plt.show()

plt.figure(figsize=(10,5))
integral = msmt.integral_fft(msmt.frequency-half_width_integral, msmt.frequency+half_width_integral)
plt.plot(range(1,msmt.number_of_pulses+1-1), integral[:-1], 'o')
plt.xlabel('Pulse number')
plt.ylabel('Integral')
plt.title('Integral of the shaded area for each pulse , timestamp : {}'.format(msmt.timestamp))
plt.show()
rel_diff = (integral.max()-integral.min())/integral.max()
print "Lowest integral is {:.2f}% lower than highest".format(rel_diff*100)
# infidelity = 
# print "This gives rise to an infidelity of {}".format()

In [ ]:
0.25*(np.pi*0.04)**2

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(msmt.freq[0], msmt.mean_fft, 'k', label='Mean')
plt.fill_between(msmt.freq[0],  msmt.mean_fft - msmt.std_fft, msmt.mean_fft + msmt.std_fft, 
                 facecolor='blue', alpha=0.5, label=r'$\pm$ 1 SD')
# set_limits()
# plot_ideal_fft()
plt.xlabel('f (Hz)')
plt.title('Mean Fourier transform of {} consecutive pulses , timestamp : {}'.format(msmt.number_of_pulses, msmt.timestamp))
plt.xlim(msmt.frequency - 1e8, msmt.frequency + 1e8)
plt.legend()
plt.show()

# plt.figure(figsize=(10,3))
# plt.plot(freq[msmt][0],  std, label="Noise on RF signal")
# # plt.plot([1.66e-9, 1.8e-9], [noise_oscilloscope, noise_oscilloscope], 'k--', label="Noise oscilloscope")
# plt.xlim(msmt.frequency - 1e8, msmt.frequency + 1e8)
# plt.ylim(0,5e-10)
# plt.xlabel('f (Hz)')
# plt.title('Standard deviation of consequtive pulses, timestamp : {}'.format(timestamp[msmt]))
# plt.show()
# ind_low = int(np.argmin(np.abs(msmt.freq[0] - 1.705e9)))
# ind_high = int(np.argmin(np.abs(msmt.freq[0] - 1.735e9)))
# mean_plateau = np.mean(mean[ind_low:ind_high])
# print "The relative mean standard deviation to the peak is {:.3e}.".format(np.mean(std)/mean_plateau)

In [ ]:
np.savetxt('after_switch_fft.csv', msmt.mean_fft, delimiter=',')

In [ ]:
np.savetxt('after_switch_freq.csv', msmt.freq[0], delimiter=',')

# Analysis of multiple consequtive measurements

In [ ]:
# read_data()
mean = np.zeros((number_of_measurements, points_per_pulse/2))
for msmt in range(number_of_measurements):
    mean[msmt] = np.mean(fft[msmt], axis=0)

plt.figure(figsize=(10,8))
plt.plot(freq[0][0], mean[0])#, label='{}'.format(msmt))
for msmt in range(number_of_measurements):
    plt.plot(freq[0][0], mean[msmt], label='{}'.format(msmt))
set_limits()
plt.title('Mean Fourier transforms of {} measurements of {} consecutive pulses, \ntimestamps : {} - {}'
          .format(number_of_measurements, number_of_pulses, timestamp[number_of_measurements-1], timestamp[0]))
plt.xlabel('f (Hz)')
plt.show()